In [1]:
import sys 
import os 
import os.path as osp 

import numpy as np 
import pandas as pd 

In [2]:
def read_excel(Path:str) -> pd.core.frame.DataFrame: 
    df = pd.read_excel(Path, index_col = 0 ) # (ref) https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html
    df = df.dropna(axis=0)  # drop rows(axis=0) including NaN

    return df

In [3]:
rootDir = "data"
targetFeature = "indiv"
categoryPath = osp.join(rootDir, targetFeature)
category_list = sorted(os.listdir(categoryPath ))
print(category_list)

action_category = category_list[0] # select a category 

['semi_indiv_category1', 'semi_indiv_category2', 'semi_indiv_category3', 'semi_indiv_category4', 'semi_indiv_category5', 'semi_indiv_category6']


In [4]:
subj_files = sorted(os.listdir(osp.join(categoryPath, action_category )))
print(subj_files)

['100_semi_indiv_category1.xlsx', '101_semi_indiv_category1.xlsx', '102_semi_indiv_category1.xlsx', '103_semi_indiv_category1.xlsx', '104_semi_indiv_category1.xlsx', '105_semi_indiv_category1.xlsx', '107_semi_indiv_category1.xlsx', '108_semi_indiv_category1.xlsx', '109_semi_indiv_category1.xlsx', '110_semi_indiv_category1.xlsx', '111_semi_indiv_category1.xlsx', '112_semi_indiv_category1.xlsx', '113_semi_indiv_category1.xlsx', '114_semi_indiv_category1.xlsx', '115_semi_indiv_category1.xlsx', '117_semi_indiv_category1.xlsx', '119_semi_indiv_category1.xlsx', '120_semi_indiv_category1.xlsx', '121_semi_indiv_category1.xlsx', '122_semi_indiv_category1.xlsx', '123_semi_indiv_category1.xlsx', '124_semi_indiv_category1.xlsx', '125_semi_indiv_category1.xlsx', '126_semi_indiv_category1.xlsx', '127_semi_indiv_category1.xlsx', '129_semi_indiv_category1.xlsx']


In [5]:
annoPath = osp.join(categoryPath, action_category, subj_files[0])
csv_df = read_excel(annoPath)
csv_df.head()

,Start_min,Start_sec,End_min,End_sec,Indiv_feature,Motion,label_ 1,label_ 2,label_ 3,label_ 4,label_ 5,label_ 6,label_ 7,label_ 8,label_ 9,label_10,label_11,label_12,label_13,label_14
Num,,,,,,,,,,,,,,,,,,,,
4,0,35,0,36,-1.613804,얼굴 찡그림,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,0,36,0,58,-1.214109,의자 흔듬,0,0,0,0,0,0,0,0,1,0,0,0,0,0
6,1,2,1,8,-0.408723,의자 흔듬,0,0,0,0,0,0,0,0,1,0,0,0,0,0
7,1,20,1,22,-0.016706,의자 흔듬,0,0,0,0,0,0,0,0,1,0,0,0,0,0
8,1,22,1,26,-0.025956,자세 바로잡기 + 얼굴 찡그림,0,0,0,0,0,0,0,0,0,1,1,0,0,0


In [6]:
print(csv_df.shape)
anno = csv_df.iloc[0][6:].tolist()

print(anno)
print(len(anno))# get one-hot vector from 

(350, 20)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
14


## Activity One-Hot

In [7]:
from sklearn.preprocessing import OneHotEncoder
# (ref) https://datamasters.co.kr/78

s = pd.Series(range(0,14))
onehot = OneHotEncoder(sparse=False)

activity_label = onehot.fit_transform(s.to_numpy().reshape(-1, 1)).astype('uint8')
print(activity_label)

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1]]


## Activity count 

In [8]:
from collections import OrderedDict

activity_dict = OrderedDict()


num, _ = csv_df.shape


for act_idx, label in enumerate(activity_label):

    activity_dict[act_idx] = []

    for i in range(num):
        anno = csv_df.iloc[i][6:].to_numpy()
            
        check_act = np.bitwise_and(label, anno)

        if np.array_equal(label, check_act): # (ref) https://numpy.org/doc/stable/reference/generated/numpy.array_equal.html
            feature = csv_df['Indiv_feature'].iloc[i]
            activity_dict[act_idx].append(feature)
    

In [9]:
count = 0

for key, val in activity_dict.items():
    count += len(val)

print(count)
print(csv_df.shape)

381
(350, 20)
